In [ ]:
import marimo as mo

# Week 13: Attention Mechanisms and Transformers**IME775: Data Driven Modeling and Optimization**📖 **Reference**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 11---## Learning Objectives- Understand the attention mechanism mathematically- Master self-attention and multi-head attention- Learn the complete Transformer architecture- Connect to modern applications (BERT, GPT)

In [ ]:
import numpy as npimport matplotlib.pyplot as plt

## 13.1 The Attention Mechanism**Core Idea**: Given a query, compute relevance scores over keys to retrieve values.$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

## 13.2 Multi-Head AttentionMultiple attention heads can capture different relationship types:$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O$$Each head: $\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$

## 13.3 Positional EncodingSelf-attention is permutation-invariant - it doesn't know position!**Solution**: Add positional information:$$PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d})$$$$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d})$$

In [ ]:
# Visualize positional encoding    pe = np.zeros((max_len, d_model))    position = np.arange(max_len)[:, np.newaxis]    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))    pe[:, 0::2] = np.sin(position * div_term)    pe[:, 1::2] = np.cos(position * div_term)    return pemax_len = 100d_pe = 64pe = positional_encoding(max_len, d_pe)fig3, axes3 = plt.subplots(1, 2, figsize=(14, 5))# Full encodingim3 = axes3[0].imshow(pe.T, aspect='auto', cmap='RdBu', vmin=-1, vmax=1)axes3[0].set_xlabel('Position')axes3[0].set_ylabel('Dimension')axes3[0].set_title('Positional Encoding Matrix')plt.colorbar(im3, ax=axes3[0])# Individual dimensionsfor dim in [0, 1, 10, 20]:    axes3[1].plot(pe[:, dim], label=f'dim {dim}', linewidth=1.5)axes3[1].set_xlabel('Position')axes3[1].set_ylabel('Encoding Value')axes3[1].set_title('Positional Encoding: Different Dimensions')axes3[1].legend()axes3[1].grid(True, alpha=0.3)plt.tight_layout()fig3

## 13.4 The Transformer Architecture```Input → Embedding + Position → [Encoder × N] → [Decoder × N] → OutputEncoder:                    Decoder:┌─────────────────┐        ┌─────────────────┐│ Multi-Head Attn │        │ Masked Self-Attn││   + Add & Norm  │        │   + Add & Norm  │├─────────────────┤        ├─────────────────┤│                 │        │ Cross-Attention ││                 │        │   + Add & Norm  │├─────────────────┤        ├─────────────────┤│ Feed-Forward    │        │ Feed-Forward    ││   + Add & Norm  │        │   + Add & Norm  │└─────────────────┘        └─────────────────┘```

In [ ]:
# Visualize Transformer architecturefig4, ax4 = plt.subplots(figsize=(14, 10))# Encoder stackenc_x = 3for i in range(3):    y_base = 2 + i * 2.5    # Multi-head attention    ax4.add_patch(plt.Rectangle((enc_x - 1, y_base), 2, 0.8,                                  facecolor='lightblue', edgecolor='black'))    ax4.text(enc_x, y_base + 0.4, 'Multi-Head\nSelf-Attention',             ha='center', va='center', fontsize=8)    # Add & Norm    ax4.add_patch(plt.Rectangle((enc_x - 1, y_base + 0.9), 2, 0.3,                                 facecolor='lightgray', edgecolor='black'))    ax4.text(enc_x, y_base + 1.05, 'Add & Norm', ha='center', va='center', fontsize=7)    # FFN    ax4.add_patch(plt.Rectangle((enc_x - 1, y_base + 1.3), 2, 0.6,                                 facecolor='lightyellow', edgecolor='black'))    ax4.text(enc_x, y_base + 1.6, 'Feed Forward', ha='center', va='center', fontsize=8)    # Add & Norm    ax4.add_patch(plt.Rectangle((enc_x - 1, y_base + 2.0), 2, 0.3,                                 facecolor='lightgray', edgecolor='black'))    ax4.text(enc_x, y_base + 2.15, 'Add & Norm', ha='center', va='center', fontsize=7)    # Residual connections    ax4.plot([enc_x - 1.3, enc_x - 1.3], [y_base + 0.4, y_base + 1.05], 'r-', lw=1)    ax4.plot([enc_x - 1.3, enc_x - 1], [y_base + 1.05, y_base + 1.05], 'r-', lw=1)# Decoder stackdec_x = 9for i in range(3):    y_base = 2 + i * 2.5    # Masked self-attention    ax4.add_patch(plt.Rectangle((dec_x - 1, y_base), 2, 0.6,                                 facecolor='lightgreen', edgecolor='black'))    ax4.text(dec_x, y_base + 0.3, 'Masked\nSelf-Attn', ha='center', va='center', fontsize=7)    # Cross-attention    ax4.add_patch(plt.Rectangle((dec_x - 1, y_base + 0.7), 2, 0.6,                                 facecolor='lightcoral', edgecolor='black'))    ax4.text(dec_x, y_base + 1.0, 'Cross\nAttention', ha='center', va='center', fontsize=7)    # FFN    ax4.add_patch(plt.Rectangle((dec_x - 1, y_base + 1.4), 2, 0.5,                                 facecolor='lightyellow', edgecolor='black'))    ax4.text(dec_x, y_base + 1.65, 'FFN', ha='center', va='center', fontsize=8)    # Arrow from encoder to cross-attention    if i == 2:        ax4.annotate('', xy=(dec_x - 1, y_base + 1.0), xytext=(enc_x + 1, y_base + 1.0),                    arrowprops=dict(arrowstyle='->', color='purple', lw=2))# Inputax4.add_patch(plt.Rectangle((enc_x - 1, 0.5), 2, 0.8, facecolor='white', edgecolor='black'))ax4.text(enc_x, 0.9, 'Input\nEmbedding\n+ Position', ha='center', va='center', fontsize=8)ax4.add_patch(plt.Rectangle((dec_x - 1, 0.5), 2, 0.8, facecolor='white', edgecolor='black'))ax4.text(dec_x, 0.9, 'Output\nEmbedding\n+ Position', ha='center', va='center', fontsize=8)# Outputax4.add_patch(plt.Rectangle((dec_x - 1, 10), 2, 0.8, facecolor='white', edgecolor='black'))ax4.text(dec_x, 10.4, 'Linear +\nSoftmax', ha='center', va='center', fontsize=8)# Labelsax4.text(enc_x, 11, 'ENCODER', ha='center', fontsize=14, fontweight='bold', color='blue')ax4.text(dec_x, 11, 'DECODER', ha='center', fontsize=14, fontweight='bold', color='green')# Arrowsax4.annotate('', xy=(enc_x, 2), xytext=(enc_x, 1.3), arrowprops=dict(arrowstyle='->', color='black'))ax4.annotate('', xy=(dec_x, 2), xytext=(dec_x, 1.3), arrowprops=dict(arrowstyle='->', color='black'))ax4.annotate('', xy=(dec_x, 10), xytext=(dec_x, 9.5), arrowprops=dict(arrowstyle='->', color='black'))ax4.set_xlim(0, 12)ax4.set_ylim(0, 12)ax4.axis('off')ax4.set_title('Transformer Architecture (Simplified)', fontsize=14, y=0.98)fig4

## 13.5 Masked Attention (Decoder)Prevent attending to future positions during training:$$\text{Mask}_{ij} = \begin{cases} 0 & i \geq j \\ -\infty & i < j \end{cases}$$

## 13.6 Modern Transformer Variants| Model | Type | Pre-training Task | Use Case ||-------|------|-------------------|----------|| **BERT** | Encoder | Masked LM + Next Sentence | Classification, NER || **GPT** | Decoder | Next Token Prediction | Generation, Chat || **T5** | Encoder-Decoder | Text-to-Text | Translation, Summarization || **ViT** | Encoder | Image Classification | Vision tasks |

In [ ]:
# Compare architecturesfig6, ax6 = plt.subplots(figsize=(12, 6))models = ['BERT-base', 'BERT-large', 'GPT-2', 'GPT-3', 'T5-base', 'ViT-base']params = [110, 340, 1500, 175000, 220, 86]  # In millionscolors = ['blue', 'blue', 'green', 'green', 'orange', 'purple']bars = ax6.bar(models, params, color=colors, alpha=0.7, edgecolor='black')ax6.set_ylabel('Parameters (Millions)')ax6.set_title('Transformer Model Sizes')ax6.set_yscale('log')# Add parameter countsfor bar, param in zip(bars, params):    height = bar.get_height()    ax6.text(bar.get_x() + bar.get_width()/2., height,            f'{param:,}M', ha='center', va='bottom', fontsize=10, fontweight='bold')# Legendax6.text(0.02, 0.95, '■ BERT (Encoder)', color='blue', transform=ax6.transAxes, fontsize=10)ax6.text(0.02, 0.90, '■ GPT (Decoder)', color='green', transform=ax6.transAxes, fontsize=10)ax6.text(0.02, 0.85, '■ T5 (Enc-Dec)', color='orange', transform=ax6.transAxes, fontsize=10)ax6.text(0.02, 0.80, '■ ViT (Vision)', color='purple', transform=ax6.transAxes, fontsize=10)ax6.grid(True, alpha=0.3, axis='y')fig6

## Summary| Component | Purpose ||-----------|---------|| **Attention** | Compute relevance-weighted combinations || **Self-Attention** | Relate positions within sequence || **Multi-Head** | Capture multiple relationship types || **Positional Encoding** | Add position information || **Masking** | Prevent attending to future |---## References- **Primary**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 11.- **Transformer**: Vaswani et al. (2017). "Attention Is All You Need."- **BERT**: Devlin et al. (2019)- **GPT-3**: Brown et al. (2020)## Connection to ML Refined CurriculumTransformers are the current state-of-the-art for:- Sequence modeling (beyond RNNs from Week 11)- Automatic feature learning- Foundation for modern AI systems